<a href="https://colab.research.google.com/github/king-gun1116/django_lec/blob/main/FIELD_Session_2_New_Republicans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#2024 FIELD 11조

In [ ]:
%pip install haversine

In [ ]:
from haversine import haversine_vector, Unit
from scipy.optimize import linprog
import numpy as np
import pandas as pd
from google.colab import files

##Part 0. 데이터 업로드

In [ ]:
# 공장, 항구, 수요지, 차종 이름: list
factory = ['HMI', 'HAOS', 'BHMC', 'HMMA', 'HMMC', 'HMB', 'HMMI', 'HTBC', 'Vietnam', 'Korea', 'HMGMA']
port = ['Seattle', 'LosAngeles', 'Houston', 'Charleston','NewJersey']
demand = ['Chicago', 'LA', 'WashingtonDC','Michigan', 'NewYork', 'California', 'Montana', 'Florida']
car = ['Elantra', 'Sonata', 'Palisade', 'Kona', 'Venue', 'Nexo', 'SantaCruz', 'IONIQ5', 'IONIQ6', 'IONIQ5Robotaxi',
        'Tucson', 'Santa-Fe', 'GV70', 'GV70EV', 'GV60', 'GV80', 'G70', 'G80(RG3)', 'G80(RG3EV)', 'G90']

In [ ]:
# 공장, 항구, 수요지의 좌표 (위도, 경도): dictionary, tuple
factory_location = {
    'HMI': (12.9992, 79.9906),
    'HAOS': (40.7724, 30.0244),
    'BHMC': (39.9689, 116.3194),
    'HMMA': (32.3887, -86.3544),
    'HMMC': (49.6624, 18.3960),
    'HMB': (-22.6848, -47.6035),
    'HMMI': (-6.404, 107.186),
    'HTBC': (31.118, 104.184),
    'Vietnam': (21.346, 105.610),
    'Korea': (36.7925, 126.9928),
    'HMGMA': (32.670, -83.2225)
}

port_location = {
    'Seattle': (47.3622, -122.1955),
    'LosAngeles': (33.7400, -118.2727),
    'Houston': (29.7426, -95.2622),
    'Charleston': (32.7812, -79.9283),
    'NewJersey': (40.6688, -74.0160)
}

demand_location = {
    'Chicago': (41.8666, -87.6240),
    'LA': (36.1495, -115.2490),
    'WashingtonDC': (38.8221, -77.0575),
    'Michigan': (44.7375, -85.5944),
    'NewYork': (40.7576, -73.7811),
    'California': (36.8192, -119.7898),
    'Montana': (45.7338, -108.6091),
    'Florida': (27.9837, -81.6790)
}

In [ ]:
# 차량 정보 (차량명, 판매 가격, 이익률)
car_info = {
    'Elantra': {'price': 20000, 'profit_margin': 0.05, 'is_EV': False},
    'Sonata': {'price': 25000, 'profit_margin': 0.06, 'is_EV': False},
    'Palisade': {'price': 35000, 'profit_margin': 0.1, 'is_EV': False},
    'Kona': {'price': 30000, 'profit_margin': 0.1, 'is_EV': True},
    'Venue': {'price': 20000, 'profit_margin': 0.05, 'is_EV': False},
    'Nexo': {'price': 60000, 'profit_margin': 0.1, 'is_EV': True},
    'SantaCruz': {'price': 28000, 'profit_margin': 0.07, 'is_EV': False},
    'IONIQ5': {'price': 45000, 'profit_margin': 0.12, 'is_EV': True},
    'IONIQ6': {'price': 47000, 'profit_margin': 0.12, 'is_EV': True},
    'IONIQ5Robotaxi': {'price': 55000, 'profit_margin': 0.12, 'is_EV': True},
    'Tucson': {'price': 27000, 'profit_margin': 0.06, 'is_EV': False},
    'Santa-Fe': {'price': 32000, 'profit_margin': 0.07, 'is_EV': False},
    'GV70': {'price': 50000, 'profit_margin': 0.12, 'is_EV': False},
    'GV70EV': {'price': 65000, 'profit_margin': 0.15, 'is_EV': True},
    'GV60': {'price': 60000, 'profit_margin': 0.15, 'is_EV': True},
    'GV80': {'price': 70000, 'profit_margin': 0.12, 'is_EV': False},
    'G70': {'price': 45000, 'profit_margin': 0.1, 'is_EV': False},
    'G80(RG3)': {'price': 50000, 'profit_margin': 0.1, 'is_EV': False},
    'G80(RG3EV)': {'price': 70000, 'profit_margin': 0.15, 'is_EV': True},
    'G90': {'price': 80000, 'profit_margin': 0.12, 'is_EV': False},
}

In [ ]:
# 공장 생산량
supply_data = {
    'HMI': {
        'Elantra': 23000, 'Sonata': 8394, 'Palisade': 17019, 'Tucson': 20593},
    'HAOS': {
        'Kona': 22587, 'Venue': 5973, 'Nexo': 70},
    'BHMC': {
        'SantaCruz': 6654, 'Sonata': 8436, 'IONIQ5': 6581, 'IONIQ6': 3070},
    'HMMA': {
        'Tucson': 25428, 'Santa-Fe': 17199, 'GV70': 4812, 'GV70EV': 828},
    'HMMC': {
        'Elantra': 27273, 'Kona': 17048, 'Tucson': 32544, 'Palisade': 17330},
    'HMB': {
        'GV60': 989, 'GV70': 3493, 'GV70EV': 698, 'GV80': 4248},
    'HMMI': {
        'Santa-Fe': 29620, 'SantaCruz': 8322, 'Sonata': 8382, 'Palisade': 10195, 'Venue': 4854},
    'HTBC': {
        'G70': 2769, 'G80(RG3)': 788, 'G80(RG3EV)': 124, 'G90': 206},
    'Vietnam': {
        'IONIQ5': 8374, 'IONIQ5Robotaxi': 18, 'IONIQ6': 2928},
    'Korea': {
        'GV80': 4063, 'G80(RG3)': 889, 'G70': 1889, 'G90': 406},
    'HMGMA': {
        "IONIQ5" : 40130, 'G80(RG3EV)': 333, 'GV70EV': 4095}
}

In [ ]:
# 수요지 수요량
demand_data = {
    'Chicago': {
        'Elantra': 10719, 'Sonata': 7576, 'Kona': 17659, 'G90': 262, 'Venue': 4008},
    'LA': {
        'Sonata': 4642, 'G70': 1050, 'Venue': 2900, 'Kona': 12209, 'Tucson': 30908, 'GV70': 2673},
    'WashingtonDC': {
        'Sonata': 714, 'Kona': 9767, 'SantaCruz': 4818, 'Nexo': 36, 'IONIQ5': 22594, 'Santa-Fe': 15658, 'Palisade': 15098, 'GV60': 346},
    'Michigan': {
        'Sonata': 1330, 'G70': 2104, 'G90': 350, 'IONIQ5Robotaxi': 18, 'Santa-Fe': 14609, 'Palisade': 10860, 'GV60': 643, 'GV70': 1704},
    'NewYork': {
        'Sonata': 6690, 'G70': 1504, 'Nexo': 34, 'IONIQ6': 2143, 'GV70': 2104, 'GV70EV': 5621, 'GV80': 2555},
    'California': {
        'Elantra': 13837, 'G80(RG3EV)': 457, 'Tucson': 22374, 'IONIQ5': 32491, 'IONIQ6': 2123, 'GV70': 1824},
    'Montana': {
        'Elantra': 17461, 'Sonata': 3145, 'G80(RG3)': 714, 'Tucson': 10568, 'SantaCruz': 4474, 'IONIQ6': 1732, 'Palisade': 7523, 'GV80': 1912},
    'Florida': {
        'Elantra': 8256, 'Sonata': 1115, 'G80(RG3)': 963, 'Venue': 3919, 'Tucson': 14715, 'SantaCruz': 5684, 'Santa-Fe': 16552, 'Palisade': 11063, 'GV80': 3844},
}

##Part 1. Haversine 함수 활용해 각 공장 -> 수요지 최소수송비용 구하기

In [ ]:
# (위도, 경도) tuple 객체만 담긴 list 생성
factory_location_list = list(factory_location.values())
port_location_list = list(port_location.values())
demand_location_list = list(demand_location.values())

In [ ]:
# factory(행)와 port(열) 조합의 [ km당 수송비용 * haversine distance(km) ] 계산한 matrix
factory_to_port = 0.04*haversine_vector(port_location_list, factory_location_list, Unit.KILOMETERS, comb=True)
# demand(행)와 port(열) 조합의 [ km당 수송비용 * haversine distance(km) ] 계산한 matrix
demand_to_port = 0.1*haversine_vector(port_location_list, demand_location_list, Unit.KILOMETERS, comb=True)

In [ ]:
# 최소수송비용 결과값을 담을 수 있는 빈 float(실수, 0.0) matrix 생성
least_cost_list = np.arange(len(factory)*len(demand))
least_cost_array = 0.0*least_cost_list.reshape(len(factory), len(demand))

# factory_to_port와 demand_to_port 수송비용 matrix 차례로 읽어나가며 수송비용 합계의 최소값 찾기
for i in range(len(factory)):
  for j in range(len(demand)):
    least_cost = factory_to_port[i][0] + demand_to_port[j][0]

    for k in range(len(port)):
      cost = factory_to_port[i][k] + demand_to_port[j][k]
      if cost < least_cost:
        least_cost = cost

    # 특정 factory, demand 조합의 최소수송비용 matrix에 결과값 저장
    least_cost_array[i][j] = least_cost

In [ ]:
# 해당 matrix를 dictionary/df 형태로 변환
least_cost_matrix = {}
for i, fact in enumerate(factory):
  least_cost_matrix[fact] = {}
  for j, dem in enumerate(demand):
    least_cost_matrix[fact][dem] = least_cost_array[i][j]
least_cost_matrix_df = pd.DataFrame(least_cost_matrix)
least_cost_matrix_df_transposed = pd.DataFrame(least_cost_matrix_df.transpose())
least_cost_matrix_df_transposed.to_csv("새로운_조건_공화당_수송비용표.csv")
files.download("새로운_조건_공화당_수송비용표.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Part 2. Linprog 함수 활용해 차종별 최소수송비용 구하기

In [ ]:
# 차종 주어졌을 때 해당 차종의 생산지와 수요지 파악
# 전기차 35% 공급/수요량 증가 + 내연기관차 20% 공급/수요량 감소

def find_supplier(car_type):
  suppliers = {}
  for supplier, type_list in supply_data.items():
    if car_type in type_list:
      suppliers[supplier] = type_list[car_type]
      if car_info[car_type]["is_EV"]:
        suppliers[supplier] *= 0.9
      else:
        suppliers[supplier] *= 1.3
  return suppliers

def find_demander(car_type):
  demanders = {}
  for demander, type_list in demand_data.items():
    if car_type in type_list:
      demanders[demander] = type_list[car_type]
      if car_info[car_type]["is_EV"]:
        demanders[demander] *= 0.9
      else:
        demanders[demander] *= 1.3
  return demanders

# 생산지 수요지 정보가 주어졌을 때 알맞은 A_eq 행렬 만들기

def s_constraint(len_s, len_d):
  output = np.zeros((len_s, len_s*len_d))
  for i in range(len_s):
    output[i] = np.arange(len_s*len_d) // len_d == i
  return output

def d_constraint(len_s, len_d):
  output = np.zeros((len_d, len_s*len_d))
  for i in range(len_d):
    output[i] = np.arange(len_s*len_d) % len_d == i
  return output

In [ ]:
# 차종이 주어졌을 때 적합한 linprog 문제 생성 및 결과 도출

def return_z(car_type):
  sup = find_supplier(car_type)
  dem = find_demander(car_type)

  sup_keys = list(sup.keys())
  dem_keys = list(dem.keys())

  len_s = len(sup_keys)
  len_d = len(dem_keys)

  cost_array = np.zeros((len_s,len_d))
  for i, supplier in enumerate(sup_keys):
    for j, demander in enumerate(dem_keys):
      cost_array[i][j] = least_cost_matrix[supplier][demander]
  cost_array = cost_array.reshape((len_s*len_d,))

  A_eq = np.concatenate((s_constraint(len_s, len_d), d_constraint(len_s, len_d)))
  b_eq = np.array(list(sup.values()) + list(dem.values())).T

  return linprog(c = cost_array, A_eq = A_eq, b_eq = b_eq).fun

In [ ]:
best_z_array = np.array(list(map(return_z, car)))

## Part 3. 순수익을 구하고 엑셀로 내보내기

In [ ]:
# 공장 위치에 따른 순이익 조정
# 미국 공장 순이익률 1% 증가, 중국 공장 순이익률 3% 감소

def find_profit(car_type):
  sup = find_supplier(car_type)
  profit = sum(sup.values()) * car_info[car_type]["price"] * car_info[car_type]["profit_margin"]
  if "BHMC" in sup.keys():
    profit -= sup["BHMC"] * car_info[car_type]["price"] * 0.03
  if "HTBC" in sup.keys():
    profit -= sup["HTBC"] * car_info[car_type]["price"] * 0.03
  if "HMMA" in sup.keys():
    profit += sup["HMMA"] * car_info[car_type]["price"] * 0.02
  if "HMGMA" in sup.keys():
    profit += sup["HMGMA"] * car_info[car_type]["price"] * 0.02
  return profit

In [ ]:
# 전기차 내연차 나누어 총 순수익 구하기
# GV60, Kona, Nexo 차량 한 대 당 순이익률 750달러 증가

total_profits = []
ev_profits = []
is_EV = []
for i in car:
    total_profits.append(find_profit(i))
    is_EV.append(car_info[i]['is_EV'])
total_profits = np.array(total_profits)
is_EV = np.array(list(map(lambda x: "전기차(EV)" if x else "전기차 아님", is_EV)))

In [ ]:
result_df = pd.DataFrame()
result_df.index = car
result_df.index.name = '차량이름'
result_df["Minimized Cost: Z"] = best_z_array
result_df["Net Profit After Transportation Cost: Net Profit - Z"] = total_profits - best_z_array
result_df["전기차 유무"] = is_EV

In [ ]:
tot = sum(result_df["Net Profit After Transportation Cost: Net Profit - Z"])
tot_ev = sum(result_df.loc[result_df["전기차 유무"] == "전기차(EV)"]["Net Profit After Transportation Cost: Net Profit - Z"])
tot_non_ev = tot - tot_ev
total_df = pd.DataFrame({"내연기관차 순이익": [None, tot_non_ev, None], "전기차 순이익": [None, tot_ev, None], "전체 순이익": [None, tot, None]}).T
total_df.columns = ["Minimized Cost: Z", "Net Profit After Transportation Cost: Net Profit - Z", "전기차 유무"]

In [ ]:
result_df = pd.concat([result_df, total_df], axis = 0)
result_df.to_excel("새로운_조건_공화당_목적함수.xlsx")
files.download("새로운_조건_공화당_목적함수.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result_df

,Minimized Cost: Z,Net Profit After Transportation Cost: Net Profit - Z,전기차 유무
Elantra,3.258699e+07,3.276791e+07,전기차 아님
Sonata,1.904216e+07,2.189614e+07,전기차 아님
Palisade,3.013011e+07,1.725451e+08,전기차 아님
Kona,1.466683e+07,9.234767e+07,전기차(EV)
Venue,7.564456e+06,6.510644e+06,전기차 아님
Nexo,2.169956e+04,3.563004e+05,전기차(EV)
SantaCruz,1.164331e+07,1.924937e+07,전기차 아님
IONIQ5,1.089227e+07,2.813302e+08,전기차(EV)
IONIQ6,2.628734e+06,2.392128e+07,전기차(EV)
IONIQ5Robotaxi,1.019811e+04,9.672189e+04,전기차(EV)
